우리가 지금까지 살펴본 예제는 스칼라 함수이거나 작은 튜퓰로 만든 상대적으로 단순한 구조를 만들어 내느 함수였다. 때로 파이썬의 변경 불가능한 이름 있는 튜플인 namedtuple을 사용해 복잡한 데이터 구조를 만들 수도 있다. 이떻게 이름 있는 튜플을 사용하고, 어떻게 그것을 만들 수 있는지에 대해 배울 것이다. 또한 상태가 있는 객체 클래스 대신 변경 불가능한 이름 있는 튜플을 사용하는 방법을 살펴본다.

객체지향 프로그래밍의 이점 중 하나는 복잡한 데이터를 점진적으로 만들어 낼 수 있는 능력이다. 어떤 측면에서 객체는 함수의 결과를 캐시해둔 것이라 할 수 있다. 그래서 객체가 함수형 디자인 패턴에도 잘 들어맞는 경우가 자주 있다. 다른 경우에 객체 패ㄷ러다임은 복잡한 계산을 포함하는 프로퍼티 메서드를 제공한다. 이 또한 함수형 설계 아이디어와 잘 들어맞는다.

하지만 어떤 경우에는 객체 클래스 정의가 복잡한 객체를 만들기 위해 상태를 활용하는 경우가 있다. 비슷한 기능을 상태가 있는 객체를 활용하지 않고 제공하는 대안에 대해 살펴본다. 상태가 있는 클래스 정의를 식별하여, 적정하거나 바른 순서로 메서드 함수 호출을 수행할 수 있도록 메타 프로퍼티를 포함시킬 것이다.

다형성 클래스 정의 바깥에서 제네릭 함수를 작성할 수 있는 몇가지 기법도 살펴본다. Callable 클래스를 활용해 다형성 클래스 계층 구조를 만들 수 있다. 이러한 구조는 함수형 설계에서 불필요한 부가 비용일 수도 있다.

### 변경 불가능한 이름 있는 튜플을 레코드로 사용하기

상황에 따라 다음 세 가지 방식 중 어느 것이든 활용할 수 있다.

* 인덱스를 사용해 이름이 붙은 원소를 선택하는 람다
* 매개변수를 사용해 인덱스에 매핑되는 매개변수 이름에 따라 원소를 선택하는 람다
* 애트리뷰트 이름이나 인덱스를 사용해 원소를 선택하는 이름 있는 튜플

튜플에서 값을 선택할 때 사용할 수 있는 세 가지 방법을 살펴보자. 첫 번째 기법은 튜플에서 인덱스를 가지고 원소를 선택하는 간단한 함수를 정의하는 것이다.

In [1]:
start = lambda leg: leg[0]
end = lambda leg: leg[1]
distance = lambda leg: leg[2]
latitude = lambda pt: pt[0]
longitude = lambda pt: pt[1]

이러한 정의가 있다면 latitude(start(first_leg))를 사용해 원하는 데이터를 참조할 수 있다.

이러한 정의는 대상 데이터 타입에 대한 정보를 그리 많이 제공하지 않는다. 이를 좀 더 분명히 하기 위해 간단한 명명 규칙을 사용할 수있다. 다음은 이름에 접두사를 붙인 선택 함수를 보여준다.

In [2]:
start_point = lambda leg: leg[0]
distance_nm = lambda leg: leg[2]
latitude_value = lambda point: point[0]

이를 주의깊게 활용하면, 이러한 명명법이 도움이 될 것이다. 이러한 방식을 너무 추구하면 각 변수  이름마다 접두사를 붙이는 헝가리 표기법처럼 복잡해질 수도 있다. 

두 번째 기법은 *매개변수 표기를 사용해 인덱스에 대한 세부 정보를 드러내는 것이다. 다음은 * 표기를 사용하는 선택 함수를 보여준다.

In [4]:
start = lambda start, end, distance: start
end = lambda start, end, distance: end
distance = lambda start, end, distance: distance
latitude = lambda lat, lon: lat
longitude = lambda lat, lon: lon

이러한 정의가 있다면, latitude(*start(*first_leg))를 사용해 데이터에서 원하는 부분을 참조할 수 있다. 이 코드는 좀 더 명확하다는 장점이 있다. 하지만 함수에 튜플을 제공하면서 *를 앞에 붙여야만 한다는 사실이 조금 이상해 보일 수 있다.

세 번째 기법은 이름 있는 튜플을 만드는 namdetuple 함수를 사용하는 것이다. 이 경우, 다음과 같이 이름 있는 튜플을 내포시켜 사용할 수 있다.

In [8]:
from collections import namedtuple
Leg = namedtuple("Leg", ("start", "end", "distance"))

이렇게 하면 first_leg.start.latitude를 사용해 데이터의 특정 부분을 가져올 수 있다. 전위 형식의 함수가 후위 형식의 애트리뷰트로 바뀐 것이 이름 있는 튜플로 쓰였다는 것을 알아챌 수 있도록 해준다. 반면, 구분이 바뀌는 것이 혼동을 초래할 수도 있다.

또한 원데이터를 처리하는 과정에서 tuple() 함수를 적당한 Leg()나 Point() 함수 호출로 변경할 것이다. 암시적으로 튜플을 만들어 내는 return이나 yield문도 찾아 바꿔줘야 한다.

경우에 따라 namedtuple 함수를 사용하면 코드가 더 명확해지기도 한다. 반면, 전위 함수에서 후위 프로퍼티로 문법만 바뀌고, 다른 실익이 없는 경우도 있다. 

### 함수형 생성자로 이름 있는 튜플 만들기

namedtuple의 인스턴스를 만드는 방법 세 가지다. 어던 기법을 선택할 것인지는 객체를 생성할 때 얼마나 많은 정보가 있느냐에 따라 달라진다.

세 기법 중 두 가지를 앞 절에서 살펴봤다. 여기서는 설계 시 고려할 점을 강조할 것이다. 설계 시에는 다음과 같은 선택 사항이 있다.

* 매개변수 값을 위치에 따라 제공할 수 있다. 평가할 식이 하나 이상 있다면 이러한 방식이 잘 작동할 것이다. Leg 객체를 만들면서 start, end에 haversiine() 함수를 적용할 때 이러한 방법을 사용했다.


* 매개변수 구문을 사용해 튜플 안에서의 위치에 따라 매개변수를 대입한다. 인자를 기존 튜플이나 반복 가능 객체로부터 기정할 경우, 이러한 방식이 잘 작동한다. map()을 사용해 float() 함수를 latitude와 longitude 값에 적용할 때 이를 사용했다.


* 키워드 대입을 활용할 수 있다. 

### 상태가 있는 클래스 사용을 피하기 위해 튜플 사용하기

지금까지 살펴본 예제에서 감싸기-풀기 디자인 패턴을 사용해 변경 불가능한 튜플이나 이름 있는 튜플을 사용할 수 있다는 것을 살펴봤다. 이러한 류의 설계에 있어 중요한 점은 변경 가능한 인스턴스 변수 대신 변경 불가능한 객체를 사용해 다른 변경 불가능한 객체를 감싸야 한다는 것이다.

